### TensorFlow-Slim image classification model library

git clone https://github.com/tensorflow/models/

**TensorFlow-Slim**은 AlexNet, VGG, ResNet, Inception과 같이 image classification에 사용되는 Deep Learning CNN 모델을 제공하고 있다. CNN 모델 뿐만 아니라 여러 Datasets(ImageNet, CIFAR-10, MNIST 등)도 제공하고 있고 이를 이용한 학습 및 실험 코드도 제공하고 있다. 그리고 각 CNN 모델을 ImageNet 데이터로 학습한 Pre-trained 모델도 제공하여 새로운 Datasets에 대한 Fine-tuning도 가능하게 해준다.

In [1]:
%matplotlib inline

import os
import sys
import math
import numpy as np
import tensorflow as tf

sys.path.insert(0,'../models/research/slim')
from nets import vgg
from datasets import cifar10

sys.path.insert(0,'../')
from utils import *
from cifar10_loader import CIFAR10_loader

slim = tf.contrib.slim

### Training Information
VGG network는 입력 영상으로 224x224의 image를 받는다. 이번 실습에서는 **CIFAR-10**을 사용하므로 num_classes를 10으로 정의한다. 매 iteration 마다 100개의 images를 학습시킬 것이므로 batch_size는 100이 된다. 총 epoch은 10회 수행하고, (**CIFAR-10** training data 50000개 기준 약 3시간 소요) 매 50번의 iteration마다 network의 parameters를 저장한다.

learning rate은 초기에 0.001로 정의하고 5 epoch마다 이 값은 1/10로 줄어든다. ImageNet으로 학습된 **VGG-16** network의 parameters는 **vgg_models/vgg_16.ckpt**이다.

#### Exercise 1-1
학습 횟수 30회, 초기 learning_rate 0.01, 매 10회 학습 마다 learning_rate은 20%로 감소

In [11]:
image_size = 224 # input image size of VGG network
num_classes = 10 # CIFAR-10 number of classes
batch_size = 100 # number of images per training iteration

num_epochs = 10 # number of training epoch
save_checkpoint_frequency = 50 # frequency of saving checkpoints
print_frequency = 50

initial_learning_rate = 0.001 # initial learning rate for optimizer
num_epochs_before_decay=5
learning_rate_decay_rate = 0.1 # decay rate for learning rate

checkpoint_path = 'vgg_models/vgg_16.ckpt' # Pre-trained VGG-16 checkpoints

### Download CIFAR-10 Dataset
**CIFAR-10** dataset은 32x32 사이즈의 이미지들로 airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck으로 이루어져있다. 각 클래스 별 6천장씩 구성되어 있으며 이중 50000장이 training에 사용되고 10000장이 test에 사용된다. TF-Slim library에서 제공하는 코드를 이용하여 현재 작업중인 폴더의 하위 폴더에 cifar-10 dataset을 저장한다.

*CIFAR-10 download link*: https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

In [3]:
data_dir = os.getcwd()
data_url = 'http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
maybe_download_and_extract(data_url, data_dir, 'cifar-10-batches')

loader = CIFAR10_loader()
class_names = loader.get_class_names()

# Model parameters
save_path = 'cifar10_checkpoints/cifar10_cnn'
if not os.path.exists('cifar10_checkpoints'): os.makedirs('cifar10_checkpoints')

Extracting Finished
50000
500


### Download pre-trained model (vgg-16)

https://github.com/tensorflow/models/tree/master/research/slim

TF-Slim github를 보면 ImageNet으로 학습된 Inception, Resnet, VGG 등 다양한 모델들의 checkpoints가 제공된다. 먼저 오늘 실습에 사용될 **VGG-16**의 checkpoint를 다운 받는다.

#### Exercise 1-2
vgg-19 network로 pre-trained model을 읽어와 학습할 수 있도록 수정

In [4]:
model_dir = 'vgg_models'
model_url = 'http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz'
maybe_download_and_extract(model_url, model_dir, 'vgg_16.ckpt')

### Create vgg-16 Model

https://github.com/tensorflow/models/blob/master/research/slim/nets/vgg.py

**VGG-16** network를 Slim library로 생성한다. 먼저 CNN에 입력으로 들어갈 영상, 클래스 명 2개의 placeholder를 정의하고, **CIFAR-10** dataset에 맞게 **VGG-16** 모델의 마지막 fc layer만 수정한다. **tf.contrib.slim.get_variables_to_restore()**을 통해 checkpoint로부터 가져 올 parameters를 정하도록 한다.

#### Exercise 1-2
vgg-19 network로 pre-trained model을 읽어와 학습할 수 있도록 수정

In [5]:
g = tf.Graph()

# input placeholders.
images = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size, image_size, 3],
                           name='images')
labels = tf.placeholder(dtype=tf.int64, shape=[batch_size], name='labels')

# Create the model, use the default arg scope to configure the batch norm parameters.
"""
activation_fn=tf.nn.relu,
weights_regularizer=slim.l2_regularizer(weight_decay),
biases_initializer=tf.zeros_initializer()
padding='SAME'
"""
with slim.arg_scope(vgg.vgg_arg_scope()):
    # 10 classes instead of 1001.
    logits, _ = vgg.vgg_16(images, num_classes=num_classes, is_training=True)
    """
    with tf.variable_scope(images,
           num_classes=num_classes,
           is_training=True,
           dropout_keep_prob=0.5,
           spatial_squeeze=True,
           scope='vgg_16',
           fc_conv_padding='VALID') as sc:
        end_points_collection = sc.name + '_end_points'
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                            outputs_collections=end_points_collection):
            logits = slim.repeat(images, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            logits = slim.max_pool2d(logits, [2, 2], scope='pool1')
            logits = slim.repeat(logits, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            logits = slim.max_pool2d(logits, [2, 2], scope='pool2')
            logits = slim.repeat(logits, 3, slim.conv2d, 256, [3, 3], scope='conv3')
            logits = slim.max_pool2d(logits, [2, 2], scope='pool3')
            logits = slim.repeat(logits, 3, slim.conv2d, 512, [3, 3], scope='conv4')
            logits = slim.max_pool2d(logits, [2, 2], scope='pool4')
            logits = slim.repeat(logits, 3, slim.conv2d, 512, [3, 3], scope='conv5')
            logits = slim.max_pool2d(logits, [2, 2], scope='pool5')
            logits = slim.conv2d(logits, 4096, [7, 7], padding=fc_conv_padding, scope='fc6')
            logits = slim.dropout(logits, dropout_keep_prob, is_training=is_training,
                             scope='dropout6')
            logits = slim.conv2d(logits, 4096, [1, 1], scope='fc7')
            logits = slim.dropout(logits, dropout_keep_prob, is_training=is_training,
                             scope='dropout7')
            logits = slim.conv2d(logits, num_classes, [1, 1],
                            activation_fn=None,
                            normalizer_fn=None,
                            scope='fc8')
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                logits = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = logits
    """
print(logits)

# Before defining remaining layers (softmax, optimizer), selecting the
# variables to be restored
exclude_layers = ['vgg_16/fc8']
#exclude_layers = ['vgg_16/fc8'] # when discaring only the last classification layer
variables_to_restore = slim.get_variables_to_restore(exclude=exclude_layers)
print('===> The list of variables to be restored:')
for i in variables_to_restore: print(i.op.name)

Tensor("vgg_16/fc8/squeezed:0", shape=(100, 10), dtype=float32)
===> The list of variables to be restored:
vgg_16/conv1/conv1_1/weights
vgg_16/conv1/conv1_1/biases
vgg_16/conv1/conv1_2/weights
vgg_16/conv1/conv1_2/biases
vgg_16/conv2/conv2_1/weights
vgg_16/conv2/conv2_1/biases
vgg_16/conv2/conv2_2/weights
vgg_16/conv2/conv2_2/biases
vgg_16/conv3/conv3_1/weights
vgg_16/conv3/conv3_1/biases
vgg_16/conv3/conv3_2/weights
vgg_16/conv3/conv3_2/biases
vgg_16/conv3/conv3_3/weights
vgg_16/conv3/conv3_3/biases
vgg_16/conv4/conv4_1/weights
vgg_16/conv4/conv4_1/biases
vgg_16/conv4/conv4_2/weights
vgg_16/conv4/conv4_2/biases
vgg_16/conv4/conv4_3/weights
vgg_16/conv4/conv4_3/biases
vgg_16/conv5/conv5_1/weights
vgg_16/conv5/conv5_1/biases
vgg_16/conv5/conv5_2/weights
vgg_16/conv5/conv5_2/biases
vgg_16/conv5/conv5_3/weights
vgg_16/conv5/conv5_3/biases
vgg_16/fc6/weights
vgg_16/fc6/biases
vgg_16/fc7/weights
vgg_16/fc7/biases


학습에 사용할 loss function과 parameters를 수정할 optimizer를 정한다. 해당 실습에서는 cross_entropy와 softmax를 loss function으로 사용하였고, Gradient Descent알고리즘을 optimizer로 사용해 학습시켰다. **tf.contrib.slim.get_variables_to_restore()**을 통해 학습시킬 레이어를 선택할 수 있다.

#### Exercise 1-3
variable_to_learn을 이용하여 앞 단의 conv layer는 고정시키고 fc layers만 학습되도록 수정

참조: https://www.tensorflow.org/api_docs/python/tf/contrib/framework/get_variables_to_restore

In [25]:
# Define the loss function
probabilities = tf.nn.softmax(logits)
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=labels, logits=logits, name='cross_entropy_per_example')

# select variables to be learned
variables_to_learn = slim.get_variables_to_restore()
#variables_to_learn = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
print('\n===> The list of variables to be learned:')
for i in variables_to_learn: print(i.op.name)

decay_step = int(num_epochs_before_decay * iteration_per_epoch)
global_step = int(num_epochs * iteration_per_epoch)

lr = tf.train.exponential_decay(
learning_rate = initial_learning_rate,
global_step = global_step,
decay_steps = decay_step,
decay_rate = learning_rate_decay_rate,
staircase = True)

# Specify the optimizer and create the train op:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss, var_list=variables_to_learn)


===> The list of variables to be learned:
vgg_16/conv1/conv1_1/weights
vgg_16/conv1/conv1_1/biases
vgg_16/conv1/conv1_2/weights
vgg_16/conv1/conv1_2/biases
vgg_16/conv2/conv2_1/weights
vgg_16/conv2/conv2_1/biases
vgg_16/conv2/conv2_2/weights
vgg_16/conv2/conv2_2/biases
vgg_16/conv3/conv3_1/weights
vgg_16/conv3/conv3_1/biases
vgg_16/conv3/conv3_2/weights
vgg_16/conv3/conv3_2/biases
vgg_16/conv3/conv3_3/weights
vgg_16/conv3/conv3_3/biases
vgg_16/conv4/conv4_1/weights
vgg_16/conv4/conv4_1/biases
vgg_16/conv4/conv4_2/weights
vgg_16/conv4/conv4_2/biases
vgg_16/conv4/conv4_3/weights
vgg_16/conv4/conv4_3/biases
vgg_16/conv5/conv5_1/weights
vgg_16/conv5/conv5_1/biases
vgg_16/conv5/conv5_2/weights
vgg_16/conv5/conv5_2/biases
vgg_16/conv5/conv5_3/weights
vgg_16/conv5/conv5_3/biases
vgg_16/fc6/weights
vgg_16/fc6/biases
vgg_16/fc7/weights
vgg_16/fc7/biases
vgg_16/fc8/weights
vgg_16/fc8/biases


### Random initialization and Training from scratch
**VGG-16** network 모델의 parameters를 Random initialization한 후 **CIFAR-10** dataset으로 학습을 진행한다.

In [ ]:
# Open the session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

iteration_per_epoch = int(math.floor(loader.get_num_train_examples() / batch_size))

# Train the model
saver = tf.train.Saver()
for ie in range(num_epochs):
    for ii in range(iteration_per_epoch):
        # Load a batch data
        batch = loader.get_batch(batch_size, 'train', (224,224))

        # Run the optimizer
        _ = sess.run([train_op], feed_dict={images:batch['images'],
                                            labels:batch['labels']})

        # Print the accuracy and loss of current batch data
        if (ii+1) % print_frequency == 0:
            batch_loss, batch_prob = sess.run([loss, probabilities], 
                                             feed_dict={images:batch['images'],
                                                        labels:batch['labels']})
            pred_labels = np.argmax(batch_prob, axis=1)
            batch_loss = np.mean(batch_loss)
            batch_acc = np.mean(np.equal(pred_labels, batch['labels']))
            print('%d Epoch %d iteration - Loss (%.3f) Accuracy (%.3f)'
                      %(ie+1, ii+1, batch_loss, batch_acc))

        # Save checkpoint
        if (ii+1) % save_checkpoint_frequency == 0:
            saver.save(sess, save_path=save_path, global_step=ie*iteration_per_epoch + ii + 1)
            print('Saved checkpoint %s_%d' % (save_path, ie*iteration_per_epoch + ii + 1))

sess.close()

### Fine-tuning VGG-16
**vgg_models/vgg_16.ckpt** 파일을 읽어와 위에서 정의한 **VGG-16** network의 parameters을 수정한 후 **CIFAR-10** dataset을 학습시킨다.

In [13]:
# Open the session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Create the saver with variables to be restored
restorer = tf.train.Saver(variables_to_restore)
restorer.restore(sess, save_path=checkpoint_path)

loader.reset()

ii = 1
ie = 1
iteration_per_epoch = int(math.floor(loader.get_num_train_examples() / batch_size))

save_path = 'cifar10_finetuning_checkpoints/cifar10_cnn'
if not os.path.exists('cifar10_finetuning_checkpoints'): os.makedirs('cifar10_finetuning_checkpoints')

# Train the model
saver = tf.train.Saver()
for ie in range(2):
    for ii in range(iteration_per_epoch):
        # Load a batch data
        batch = loader.get_batch(batch_size, 'train', (224,224))

        # Run the optimizer
        _ = sess.run([train_op], feed_dict={images:batch['images'],
                                            labels:batch['labels']})

        # Print the accuracy and loss of current batch data
        if (ii+1) % print_frequency == 0:
            batch_loss, batch_prob = sess.run([loss, probabilities], 
                                             feed_dict={images:batch['images'],
                                                        labels:batch['labels']})
            pred_labels = np.argmax(batch_prob, axis=1)
            batch_loss = np.mean(batch_loss)
            batch_acc =np.mean(np.equal(pred_labels, batch['labels']))
            print('%d Epoch %d iteration - Loss (%.3f) Accuracy (%.3f)'
                      %(ie+1, ii+1, batch_loss, batch_acc))

        # Save checkpoint
        if (ii+1) % save_checkpoint_frequency == 0:
            saver.save(sess, save_path=save_path, global_step=ie*iteration_per_epoch + ii + 1)
            print('Saved checkpoint %s_%d' % (save_path, ie*iteration_per_epoch + ii + 1))
sess.close()

INFO:tensorflow:Restoring parameters from vgg_models/vgg_16.ckpt
1 Epoch 50 iteration - Loss (1.607) Accuracy (0.470)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_50
1 Epoch 100 iteration - Loss (0.976) Accuracy (0.660)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_100
1 Epoch 150 iteration - Loss (1.113) Accuracy (0.600)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_150
1 Epoch 200 iteration - Loss (0.903) Accuracy (0.680)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_200
1 Epoch 250 iteration - Loss (0.540) Accuracy (0.810)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_250
1 Epoch 300 iteration - Loss (0.434) Accuracy (0.840)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_300
1 Epoch 350 iteration - Loss (0.580) Accuracy (0.820)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_cnn_350
1 Epoch 400 iteration - Loss (0.474) Accuracy (0.820)
Saved checkpoint cifar10_finetuning_checkpoints/cifar10_c

### Evaluate the model

In [24]:
iteration_per_epoch = int(math.floor(loader.get_num_test_examples() / batch_size))
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir='cifar10_finetuning_checkpoints/')
print('Last checkpoint path is %s' % (checkpoint_path))

print(iteration_per_epoch)

# Open the session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

# Load the checkpoint or initialize the variables
saver.restore(sess, save_path=checkpoint_path)
print('Model is restored from %s' % checkpoint_path)

loader.reset()
ii = 1
num_correct = 0
num_examples = 0

# Evaluate the model
while True:
    # Load a batch data
    batch = loader.get_batch(batch_size, 'test', (224,224))
    if batch['wrapped']: break

    # Compute the correct numbers
    batch_prob = sess.run(probabilities, feed_dict={images:batch['images'],
                                                      labels:batch['labels']})

    pred_labels = np.argmax(batch_prob, axis=1)
    batch_correct_num = np.sum(np.equal(pred_labels, batch['labels']))

    num_correct += batch_correct_num
    num_examples += batch_size
    
    if (ii+1) % 10 == 0:
        print('%d/%d done' % (ii+1, iteration_per_epoch))
    ii += 1
print('Test accuracy: %.2f%%' % (float(num_correct) / float(num_examples) * 100.0))

sess.close()

Last checkpoint path is cifar10_finetuning_checkpoints/cifar10_cnn-1000
100
INFO:tensorflow:Restoring parameters from cifar10_finetuning_checkpoints/cifar10_cnn-1000
Model is restored from cifar10_finetuning_checkpoints/cifar10_cnn-1000
10/100 done
20/100 done
30/100 done
40/100 done
50/100 done
60/100 done
70/100 done
80/100 done
90/100 done
100/100 done
Test accuracy: 85.33%


### Exercise 1
위 코드를 다음과 같은 내용이 될 수 있도록 수정하도록 한다.
1. 학습 횟수 30회, 초기 learning_rate 0.01, 매 10회 학습 마다 learning_rate은 20%로 감소
2. vgg-19 network로 pre-trained model을 읽어와 학습할 수 있도록 수정
3. variable_to_learn을 이용하여 앞 단의 conv layer는 고정시키고 fc layers만 학습되도록 수정

### Exercise 2
아래 코드는 **/models/research/slim/nets/vgg.py**에 정의된 vgg-16, vgg-19 네트워크의 정의이다. vgg network의 구조와 아래 코드를 참고하여 vgg-postech 네트워크를 구현해보록 한다.

![VGG-network](../vgg-network.jpg)


In [ ]:
def vgg_arg_scope(weight_decay=0.0005):
  """Defines the VGG arg scope.

  Args:
    weight_decay: The l2 regularization coefficient.

  Returns:
    An arg_scope.
  """
  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.relu,
                      weights_regularizer=slim.l2_regularizer(weight_decay),
                      biases_initializer=tf.zeros_initializer()):
    with slim.arg_scope([slim.conv2d], padding='SAME') as arg_sc:
      return arg_sc

def vgg_16(inputs,
           num_classes=1000,
           is_training=True,
           dropout_keep_prob=0.5,
           spatial_squeeze=True,
           scope='vgg_16',
           fc_conv_padding='VALID'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                            outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2, 2], scope='pool5')
            # Use conv2d instead of fully_connected layers.
            net = slim.conv2d(net, 4096, [7, 7], padding=fc_conv_padding, scope='fc6')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                             scope='dropout6')
            net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                             scope='dropout7')
            net = slim.conv2d(net, num_classes, [1, 1],
                            activation_fn=None,
                            normalizer_fn=None,
                            scope='fc8')
            
            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = net
            return net, end_points

def vgg_19(inputs,
           num_classes=1000,
           is_training=True,
           dropout_keep_prob=0.5,
           spatial_squeeze=True,
           scope='vgg_19',
           fc_conv_padding='VALID'):

    with tf.variable_scope(scope, 'vgg_19', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                            outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 4, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2, 2], scope='pool5')
            # Use conv2d instead of fully_connected layers.
            net = slim.conv2d(net, 4096, [7, 7], padding=fc_conv_padding, scope='fc6')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                               scope='dropout6')
            net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                               scope='dropout7')
            net = slim.conv2d(net, num_classes, [1, 1],
                              activation_fn=None,
                              normalizer_fn=None,
                              scope='fc8')
            
            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = net
            return net, end_points
        
        
#TODO: implement VGG-POSTECH network
def vgg_postech(inputs,
           num_classes=1000,
           is_training=True,
           dropout_keep_prob=0.5,
           spatial_squeeze=True,
           scope='vgg_postech',
           fc_conv_padding='VALID'):
    with tf.variable_scope(scope, 'vgg_postech', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        #TODO : implement VGG-postech
        
        
        
        # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = net
            return net, end_points